In [5]:
import pandas as pd
import torch
import tensorflow as tf
from sklearn.linear_model import LinearRegression
import numpy as np
import matplotlib.pyplot as plt

In [6]:
import pandas as pd
df = pd.read_csv("dataset.csv.zip")  # Replace with actual filename
df.head()

,id,file_path,file_size,line_count,extension,language
0,1,Markdown/000001.md,34784,572,md,Markdown
1,2,XML/000002.props,3013,44,props,XML
2,3,Text/000003.txt,1076,21,txt,Text
3,4,Markdown/000004.md,8105,84,md,Markdown
4,5,Markdown/000005.md,2763,41,md,Markdown


 # STEP 2 : Unzip the Code Files

In [3]:
import zipfile
with zipfile.ZipFile("dataset.zip", "r") as zip_ref:
    zip_ref.extractall("code_files")

# STEP 3 : Read Code from File Paths


In [ ]:
import os

def read_code(path):
    full_path = os.path.join("code_files", path)
    try:
        with open(full_path, "r", encoding="utf-8") as f:
            return f.read()
    except Exception as e:
        return f"ERROR: {e}"

df["code"] = df["file_path"].apply(read_code)

In [ ]:
df

In [ ]:
df.loc[1,'code']

 # THE CLEAN FUNCTION

In [ ]:
import re
def preprocess_code(text):
        """Clean and preprocess code snippets"""
        if pd.isna(text):
            return ""

        # Convert to lowercase
        text = text.lower()

        # Remove comments
        text = re.sub(r'#.*', '', text)  # Python comments
        text = re.sub(r'//.*', '', text)  # Single-line comments
        text = re.sub(r'/\*.*?\*/', '', text, flags=re.DOTALL)  # Multi-line comments

        # Remove extra whitespace but keep basic structure
        text = re.sub(r'\s+', ' ', text)

        text = text[:10000]

        
        
        return text.strip()

In [ ]:
# import re

# def preprocess_code_for_classification(code):
#     """Clean code for classification: remove strings, normalize whitespace, preserve symbols and structure"""
#     if not isinstance(code, str):
#         return ""

#     # Remove string literals (single and double quotes)
#     code = re.sub(r'(["\'])(?:(?=(\\?))\2.)*?\1', '', code)

#     # Normalize tabs and spaces (preserve newlines)
#     code = re.sub(r'[ \t]+', ' ', code)      # Collapse multiple spaces/tabs
#     code = re.sub(r'\n+', '\n', code)        # Collapse multiple newlines

#     # Strip leading/trailing whitespace from each line
#     code = '\n'.join(line.strip() for line in code.splitlines())

#     # Truncate long files (optional)
#     code = code[:10000]

#     return code.strip()

In [ ]:
df["clean_code"] = df["code"].apply(preprocess_code)

In [ ]:
missing = df["clean_code"].isnull().sum()
print(f"Missing code rows: {missing}")

In [ ]:
df["clean_line_count"] = df["clean_code"].apply(lambda x: len(x.splitlines()))
df["clean_size"] = df["clean_code"].apply(lambda x: len(x.encode("utf-8")))

In [ ]:
df

In [ ]:
print("Rows:", len(df))
print("\nLanguage distribution:\n", df["language"].value_counts())

In [ ]:
df[["language", group_key]]

In [ ]:
import pandas as pd
print(pd.__version__)

In [ ]:
def downsample_dataframe(df, label_col="language", target_size=5000, enable=True):
    if not enable:
        return df

    def sample_group(group):
        return group.sample(n=target_size, random_state=1) if len(group) >= target_size else group

    return (
        df.groupby(label_col, group_keys=False)
          .apply(sample_group, include_group=False)
          .reset_index(drop=True)
    )

In [ ]:
df = downsample_dataframe(df, label_col="language", target_size=5000, enable=True)

In [ ]:
print(df["language"].value_counts())

In [ ]:
min_count = 5  # threshold
counts = df["language"].value_counts()

df["language"] = df["language"].apply(
    lambda x: x if counts[x] >= min_count else "Other"
)

print(df["language"].value_counts())

In [ ]:
other_count = (df["language"] == "Other").sum()
print("Number of code snippets in the otherclass:", other_count)
